In [1]:
## True if you want to import data downloaded previusly
# flag_loading2 = False

## History Prices Interval:
## 1 = 5m
## 2 = 1h
## 3 = 1d

inter2 = 3

# Update history dispite it was downloaded today
flag_history_download2 = False

In [2]:
%run 2_STRATEGY_UPDATE.ipynb


1. HISTORY

---------------
1.1 Header
---------------

Not loading updated files
Loading outdated files
Updating History Data, only once today
History interval: 1 DAY
Last operations file loaded
Last strategy file loaded
Market = portfolio_world_index_crypto_currencies_top1000_mcapT22022

---------------
1.2 Files Loading
---------------

HISTORY loaded
flag_history: False
GRAL DATA file loaded
Oudated STATS loaded
Oudated STATS_VAL loaded
Oudated EARNINGS loaded

---------------
1.3 Data Download
---------------

Stats data already gotten or not needed
Stats Val data already gotten or not needed
General data already gotten
EUR-USD downloaded
History data already gotten
Earnings data already gotten or no needed

---------------
1.4 Data Wrangling
---------------

Data already checked
Earnings data already edited or there are no earnings

---------------
1.6 Export
---------------

Nothing new to export
Nothing new to export
Nothing new to export
Nothing new to export
Nothing new to e

### 2. Analysis

In [3]:
print("""
3. TECNICAL ANALYSIS

---------------
3.1 Calculations
---------------
""")


3. TECNICAL ANALYSIS

---------------
3.1 Calculations
---------------



#### 2.1 Import

In [4]:
from sklearn.preprocessing import normalize

#### 2.2 Functions

In [5]:
from IPython.core.display import HTML
HTML("""
<style>
.p-Widget.jp-RenderedImage.jp-mod-trusted.jp-OutputArea-output {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

#### 2.3 Calculations

##### 2.3.1 General

In [6]:
analysis_ticks = strategy_df.index
analysis_df = data.loc[:,(analysis_ticks,slice(None))]

In [7]:
# SHARES NORMALIZATION (SUM = 1)
shares_norm = normalize(strategy_df["shares"].values.reshape(-1,1), axis = 0,norm="l1").round(3).flatten()

##### 2.3.2 Moving Averages Steps Optimization for strategy ticks

In [8]:
# start = -180
# t = "d"
# sma1 = range(2,20,4)
# sma2 = range(20,100,10)
           
# data2 = analysis_df.resample(f"1{t}").mean().loc[:,(analysis_ticks,"Close")][start:]
# # data2.columns = data2.columns.droplevel(1)

# results = pd.DataFrame()

# for SMA1, SMA2 in product(sma1, sma2):
#     for tic in analysis_ticks:
#         data2[tic].dropna(how="all", inplace=True)
#         data2[(tic,'Returns')] = data2[tic]['Close'].pct_change()
#         data2[(tic,'SMA1')] = data2[(tic,'Returns')].rolling(SMA1).mean()
#         data2[(tic,'SMA2')] = data2[(tic,'Returns')].rolling(SMA2).mean()
#         data2[(tic,'Position')] = np.where(data2[(tic,'SMA1')] > data2[(tic,'SMA2')], 1, -1)
#         data2[(tic,'Strategy')] = data2[(tic,'Position')].shift(1) * data2[(tic,'Returns')]
#         data2[tic].dropna(how="all", inplace=True)
        
#         perf = np.exp(data2[[(tic,'Returns'), (tic,'Strategy')]].sum())
        
#         results = results.append(pd.DataFrame(
#                     {"Ticker":tic,'SMA1': SMA1, 'SMA2': SMA2,
#                      'MARKET': perf[(tic,'Returns')],
#                      'STRATEGY': perf[(tic,'Strategy')],
#                      'OUT': perf[(tic,'Strategy')] - perf[(tic,'Returns')]},
#                      index=[0]), ignore_index=True)

# del data2, perf

In [9]:
if inter2 == 3:
    start2 = -180
else:
    start2 = -30

t = "d"
sma1 = range(2,20,4)
sma2 = range(20,100,10)
           
data2 = analysis_df.resample(f"1{t}").mean().loc[:,(analysis_ticks,"Close")][start2:]
results_analysis_ticks = pd.DataFrame()
broker_fee = 0

for SMA1, SMA2 in product(sma1, sma2):
    for tic in analysis_ticks:
        data2[tic].dropna(how="all", inplace=True)
        data2[(tic,'Returns')] = data2[tic]['Close'].pct_change()
        data2[(tic,'SMA1')] = data2[(tic,'Returns')].rolling(SMA1).mean()
        data2[(tic,'SMA2')] = data2[(tic,'Returns')].rolling(SMA2).mean()
        data2[(tic,'Position')] = np.where(data2[(tic,'SMA1')] > data2[(tic,'SMA2')], 1, 0)
        data2[(tic,'Position_ma4')] = data2[(tic,'Position')].rolling(4).mean().round()
        
        data2[(tic,'flag_op')] = np.where(data2[(tic,'Position_ma4')].shift(1).eq(1) & data2[(tic,'Position_ma4')].eq(0),
                                                       "sell",
                                                        np.where(data2[(tic,'Position_ma4')].shift(1).eq(0) & data2[(tic,'Position_ma4')].eq(1),
                                                                "buy",
                                                                ""))
        
        sell_ind = data2[tic][data2[tic]["flag_op"] == "sell"].index
        buy_ind = data2[tic][data2[tic]["flag_op"] == "buy"].index
        buy_sell_df = data2[tic][data2[tic]["flag_op"] != ""]
        
        data2[(tic,'Rets_Strat')] = 0
        # PERCENTAGE RETURNS WHEN SELLING
        data2[(tic,'Rets_Strat')][sell_ind] = np.where(data2[tic].loc[sell_ind].index,
                ((buy_sell_df["Close"] - buy_sell_df["Close"].shift(1)) / buy_sell_df["Close"].shift(1))[sell_ind],
                0)
        # BROKER FEE WHEN BUYING
        data2[(tic,'Rets_Strat')][buy_ind] = broker_fee
        
        data2[tic].dropna(how="all", inplace=True)
        
        donot_ret =  (data2[tic]["Close"][-1] - data2[tic]["Close"][start2] ) / data2[tic]["Close"][start2]
        
        results_analysis_ticks = results_analysis_ticks.append(pd.DataFrame(
                    {"ticker":tic,'SMA1': SMA1, 'SMA2': SMA2,
                     'MARKET': donot_ret,
                     'OUT': data2[(tic,'Rets_Strat')][sell_ind].sum()},
                     index=[0]), ignore_index=True)

results_analysis_ticks = results_analysis_ticks.sort_values(by="OUT", ascending = False).reset_index(drop = True)        
index_max_out = results_analysis_ticks.iloc[results_analysis_ticks.groupby(["ticker"])["OUT"].idxmax()].index
results_analysis_ticks_best = results_analysis_ticks.iloc[index_max_out].sort_values(by="OUT", ascending = False).reset_index(drop = True)

del data2, sell_ind, buy_ind, buy_sell_df, broker_fee, donot_ret, index_max_out

##### 2.3.3 Demo

In [10]:
# fig, ax = plt.subplots(3,1,figsize=(20,10))

# t = "AAPL"
# j = -160
# k = -1

# data_smopt_allticks[t]["Close"][j:k].plot(ax=ax[0],c="k", label = "Price")
# ax2 = ax[0].twinx()

# data_smopt_allticks[t]["SMA1"][j:k].plot(ax=ax2,c="g", label = "SMA1")
# data_smopt_allticks[t]["SMA2"][j:k].plot(ax=ax2,c="y", label = "SMA2")

# data_smopt_allticks[t]["Returns"][j:k].plot(ax=ax[1],c="c", label = "Returns")

# # data_smopt_allticks[t]["Strategy"][j:k].plot(ax=ax[1],c="b", label = "Strategy")
# ax[1].grid(False)

# ax3 = ax[1].twinx()
# ax3.plot(data_smopt_allticks[t]["Position"][j:k],"ro", label = "Position")

# ax[0].legend(loc="upper right")
# ax[1].legend()
# ax2.legend(loc="upper left")

# ax2.grid(False)

# strat_ma_cum_filt[t][j:k].plot(ax=ax[2],c="r", label = "Stragety")
# ax[2].legend()

# plt.show()

In [11]:
# # BUY-SELL FLAG SMOOTH WITH MOVING AVERAGE DEMO
# z = np.random.randint(1, high = 1100)
# data_smopt_allticks[(ticks[z],'Position_ma4')] = data_smopt_allticks[(ticks[z],'Position')].rolling(4).mean().round()
# plt.plot(data_smopt_allticks[ticks[z],"Position"][-100:],"or")
# data_smopt_allticks[ticks[z],"Position_ma4"][-100:].plot(c="g")
# plt.show()

##### 2.3.4 Portfolio

In [12]:
# t = "d"
t = interval[-1]

returns = analysis_df.resample(f'1{t}').mean().loc[:,(analysis_ticks,"Close")].pct_change()

returns_cartera = returns.dropna(how="all")
volatility_cartera_tick = (returns_cartera.std()*100).round(2)
volatility_cartera = np.average(volatility_cartera_tick, axis = 0, weights = shares_norm).round(2)

##### 2.3.5 VOLUME DELTA USING DIFF (IN PROGRESS)

### 3. Charts

#### 3.1 Stock Price | Volume per tick

In [13]:
#CHART 1: Stock Price | Volume per tick
def lines_plot(start4, t):
    ticks = analysis_ticks
    letra = "consolas"
    
    n = 2
    n_rows = np.ceil(len(ticks)/n).astype(int)
    n_cols = n
    ysize = len(ticks) / 0.4
    i = 0
    if t == "m":
        data2 = data
    else:
        data2 = data.resample(f'1{t}').mean()[start4:]

    if t == "d":
        width = 1
    else:
        width = 1 / 24
        
    fig, axes = plt.subplots(n_rows, n_cols, figsize = (20,ysize))
    for row_num in range(n_rows):
        for col_num in range(n_cols):
            if i <= len(analysis_ticks)-1:
                ax = axes[row_num][col_num]
                tic_name = data_gral["name"][data_gral["ticker"] == ticks[i]].values[0]
                # data2 = data[ticks[i]].groupby([pd.Grouper(freq=f'1{t}', sort=True)]).mean().interpolate()[start4:].dropna(how="all", axis=0)
                
                up = data2[ticks[i]][data2.loc[:,(ticks[i],"Close")] >= data2.loc[:,(ticks[i],"Open")]]
                down = data2[ticks[i]][data2.loc[:,(ticks[i],"Close")] < data2.loc[:,(ticks[i],"Open")]]
                
                #Price plot
                ax.fill_between(data2[ticks[i]]["Close"].index,
                                 data2[ticks[i]]["Low"], 
                                 data2[ticks[i]]["High"],
                                 color = "blue",
                                 alpha = 0.1,
                                 label = "Price delta step")
                ax.plot(data2[ticks[i]]["Close"],
                        "--k", label = "Close Price")
                
                ax.set_title(tic_name,font = letra, fontsize=20)
                ax.legend(loc=(0.01,0.8))
                # ax.set_xticklabels(ax.get_xticklabels(), rotation = 45)
                ax.set_xticklabels(ax.get_xticklabels(), rotation = 45)
                
                ax.set_ylabel("Price",font = letra, fontsize=15)
                
                if up.Volume.isnull().all() == False:
                    ax.set_ylim(np.min(data2[ticks[i]]["Close"])/1.6,np.max(data2[ticks[i]]["Close"])*1.1)
                else:
                    ax.set_ylim(np.min(data2[ticks[i]]["Close"])/1.05,np.max(data2[ticks[i]]["Close"])*1.05)
                
                # Operations reference
                try:
                    for j in range(len(operations_df.index[operations_df["ticker"] == ticks[i]])):
                        # if operations_df.index[operations_df["ticker"] == ticks[i]][j] >= min(data2[ticks[i]]["Close"].index).tz_convert(None):
                        if operations_df.index[operations_df["ticker"] == ticks[i]][j] >= min(data2[ticks[i]]["Close"].index):
                                
                            ax.axvline(operations_df.index[operations_df["ticker"] == ticks[i]][j],
                                               color="orange", linestyle="dashed", alpha = 0.6)
                            
                            ax.axhline(operations_df["operation_price"][operations_df["ticker"] == ticks[i]][j],
                                               color="orange", linestyle="dashed", alpha = 0.6)
                except:
                    pass
                
                if up.Volume.isnull().all() == False:
                    # Volume plot
                    ax2 = ax.twinx()
                    ax2.bar(up.index,up.Volume, width, color='green', edgecolor="#EAEAF2")
                    ax2.bar(down.index,down.Volume,width,color='red', edgecolor="#EAEAF2")
                    ax2.set_ylim(0,2*np.max(data2[ticks[i]].Volume))
                    
                    ax2.set_ylabel("Volume",font = letra, fontsize=15)
                    ax2.grid(False)
                else:
                    pass
                i+=1
    fig.suptitle(f'Quick Look last {-start4} days',font = letra, fontsize= 35, y = 1.001)
    fig.tight_layout()
    plt.show()


# DISPLAY
if inter2 == 3:
    start4 = widgets.IntSlider(min = -len(data.resample(f'1d')), max = -10, step = 30, value = -365, description = "Steps before",
                                      style = {'description_width': 'initial'},
                                     layout=widgets.Layout(width="30%"))
elif inter2 == 2:
    start4 = widgets.IntSlider(min = -len(data.resample(f'1h')), max = -1, step = 1, value = -5, description = "Steps before",
                                      style = {'description_width': 'initial'},
                                     layout=widgets.Layout(width="30%"))
else:
    start4 = widgets.IntSlider(min = -len(data.resample(f'5m')), max = -1, step = 1, value = -1, description = "Steps before",
                                      style = {'description_width': 'initial'},
                                     layout=widgets.Layout(width="30%"))

t = widgets.Dropdown(options = ["d","h","m"], description = "Select Prices Interval",
                     style = {'description_width': 'initial'}, layout=widgets.Layout(width="25%"))

ui = widgets.Box([start4,t])
out = widgets.interactive_output(lines_plot, {'start4': start4, 't': t})
display(ui,out)

Box(children=(IntSlider(value=-365, description='Steps before', layout=Layout(width='30%'), max=-10, min=-2194…

Output()

#### 3.2 Dashboard 1

In [14]:
print("""
---------------
2.2 DASHBOARD
---------------
""")


---------------
2.2 DASHBOARD
---------------



In [50]:
def ma_cb_plot(Ticker, start5, step1, step2 ,t):
    name = data_gral["name"][data_gral["ticker"] == Ticker].values[0]
    letra = "consolas"
    ax = True
    fig = plt.figure(figsize=(20,11), linewidth = 5, edgecolor="#BFBFBF")
    fig.patch.set_facecolor('#00193E')
    data2 = analysis_df.resample(f'1{t}').mean()[start5:]
    
    # Channels
    CH1MI = data2[Ticker]['Close'].rolling(step1).min()
    CH2MI = data2[Ticker]['Close'].rolling(step2).min()
    CH1MA = data2[Ticker]['Close'].rolling(step1).max()
    CH2MA = data2[Ticker]['Close'].rolling(step2).max()
    
    # Simpe Moving Averages
    SMA1 = data2[Ticker]['Close'].rolling(step1).mean()
    SMA2 = data2[Ticker]['Close'].rolling(step2).mean()
    
    best_sma = results_analysis_ticks[(results_analysis_ticks["ticker"] == Ticker) & (results_analysis_ticks["OUT"] > np.percentile(results_analysis_ticks[results_analysis_ticks["ticker"] == Ticker]["OUT"], 
               80))].sort_values("OUT", ascending = False)[["SMA1","SMA2","OUT"]].iloc[:4,:].T.round(2)
   
    returns = data2[Ticker]['Close'].pct_change()
    
    # volat = np.round(data2.loc[:,(Ticker,'Close')].pct_change().describe().iloc[2]*100,1)
    volat = np.round(returns.std()*100,2)    
    volat_total = np.round(analysis_df.resample(f'1{t}').mean().loc[:,(Ticker,'Close')].pct_change().std()*100,2)
    
    if t == "d":
        width = 1
    else:
        width = 1 / 24

    if ax:
        ax1 = plt.subplot(4,4,(1,7))
        ax2 = plt.subplot(4,4,(9,15))
        ax3 = plt.subplot(4,4,4)
        ax4 = plt.subplot(4,4,8)
        ax5 = plt.subplot(4,4,12)
        ax6 = plt.subplot(4,4,16)

        ax1.tick_params(colors = 'white')
        ax2.tick_params(colors = 'white')
        ax3.tick_params(colors = 'white')
        ax4.tick_params(colors = 'white')
        ax5.tick_params(colors = 'white')
        ax6.tick_params(colors = 'white')
    
    if ax1:
        ax1.set_title(f"Channel Breakinig {step1} - {step2} Days  |  {name}", 
                      font = letra,
                      fontweight = "bold",
                      fontsize=32,
                      color = "white",
                      pad = 20)
        
        ax1.set_ylabel("Close Price", font = letra, color = "white")
        ax1.plot(data2[(Ticker,"Close")], label="Close", linewidth=3)

        ax1.plot(CH1MI, "--r", label = f"MIN_CH{step1}")
        ax1.plot(CH2MI, "r", label = f"MIN_CH{step2}")
        
        ax1.plot(CH1MA, "--g", label = f"MAX_CH{step1}")
        ax1.plot(CH2MA, "g", label = f"MAX_CH{step2}")
        
        ax1.legend()
        
    if ax2:
        ax2.set_title(f"Moving Average {step1} - {step2} Days  |  {name}",
                      font = letra,
                      fontweight = "bold",
                      fontsize=32,
                      color = "white",
                      pad = 20)
        
        ax2.set_ylabel("Close Price", font = letra, color = "white")   
        
        ax2.plot(data2[(Ticker,"Close")], label="Close", linewidth=3)
        ax2.fill_between(SMA2.index,
                         SMA2,
                         SMA1,
                        alpha = 0.2,
                        color = "green",
                        where = SMA2 < SMA1)
        
        ax2.fill_between(SMA2.index,
                         SMA2,
                         SMA1,
                        alpha = 0.2,
                        color = "red",
                        where = SMA2 > SMA1)
        
        ax2.plot(SMA1, "--k", label=f"MA_{step1}")
        ax2.plot(SMA2, "k", label=f"MA_{step2}")
        ax2.axvline(data2[Ticker][-30:].index[0], c="gray", linestyle = "dotted", linewidth = 1)
        ax2.legend()
        
        # ax2.annotate("Last 365 days",xy=(data2[(Ticker,"Close")].index.min(),data2[(Ticker,"Close")].min()-8), annotation_clip = False,
        #             color = "w", size = 14)
        
        #bbox=[x coord, y coord, ancho, alto]        
        table = ax2.table(cellText = best_sma.values, rowLabels = best_sma.index, colLabels = ["Last 365 days","","","",""],
                          cellLoc="center", bbox=[0.25, -0.3, 0.6, 0.2], alpha = .8,
                         cellColours = [["#EAEAF2","#EAEAF2","#EAEAF2","#EAEAF2"],
                                        ["#EAEAF2","#EAEAF2","#EAEAF2","#EAEAF2"],
                                        ["#EAEAF2","#EAEAF2","#EAEAF2","#EAEAF2"]],
                         rowColours = ["#EAEAF2","#EAEAF2","#EAEAF2"],
                         colColours = ["#EAEAF2",'#00193E','#00193E','#00193E'])
        
        table.auto_set_font_size(False)
        table.set_fontsize(12)

        try:
            for j in range(len(operations_df.index[operations_df["ticker"] == Ticker])):
                if operations_df.index[operations_df["ticker"] == Ticker][j] >= min(data2[Ticker]["Close"].index):
                    ax2.axvline(operations_df.index[operations_df["ticker"] == Ticker][j],
                                       color="orange", linestyle="dashed", alpha = 0.6)
    
                    ax2.axhline(operations_df["operation_price"][operations_df["ticker"] == Ticker][j],
                                       color="orange", linestyle="dashed", alpha = 0.6)
        except:
            pass
       
    if ax3:
#         sns.distplot(returns, bins = 25, ax = ax4)
#         ax4.set_xlim(returns.mean() - 2*returns.std(), 
#                  returns.mean() + 2*returns.std())
       
#         ax4.set_xlabel("Returns (%)", color = "white")
#         # ax4.set_ylabel("Density", color = "white")
#         ax4.patch.set_facecolor('#00193E')
#         ax4.set_yticks([])
#         ax4.spines[:].set_visible(False)
#         ax4.grid(False)
        ax3.annotate(f"€ {np.int(data2[Ticker]['Close'][-1])}",
                     annotation_clip = False, xy=(0.05,0.35),
                     font = letra,
                     fontsize = 50,
                     color = "white",multialignment = "center",
                     alpha = 0.8,
                     path_effects=[pe.withStroke(linewidth = 4, foreground = "white",alpha = 0.2)])
        if returns[-1] >= 0:
            ax3.annotate(f"{returns[-1].round(2)} %", annotation_clip = False, xy=(0.05,-0.25),
                         font = letra,
                         fontsize = 40,
                         color = "green",multialignment = "center",
                         path_effects=[pe.withStroke(linewidth = 4, foreground = "white",alpha = 0.2)])
        else:
            ax3.annotate(f"{returns[-1].round(2)} %", annotation_clip = False, xy=(0.05,-0.25),
                         font = letra,
                         fontsize = 40,
                         color = "red",multialignment = "center",
                         path_effects=[pe.withStroke(linewidth = 4, foreground = "white",alpha = 0.2)])
            
        ax3.set_title("Current Price", font = letra, fontweight = "bold", fontsize=28, color = "white", y = 0.9)
        ax3.set_axis_off()
        
    if ax4:
        if strategy_df[strategy_df.index == Ticker]['returns_(eur)'][0] >= 0:
            
            ax4.annotate(f"{strategy_df[strategy_df.index == Ticker]['returns_(%)'][0]} %",
                         annotation_clip = False, xy=(0.05,-0.25),
                         font = letra,
                         fontsize = 40,
                         color = "green",multialignment = "center",
                         alpha = 0.8,
                         path_effects=[pe.withStroke(linewidth = 4, foreground = "white",alpha = 0.2)])
            
            ax4.annotate(f"€ {int(strategy_df[strategy_df.index == Ticker]['returns_(eur)'][0])}", annotation_clip = False, xy=(0.05,0.35),
                         font = letra,
                         fontsize = 50,
                         color = "green",multialignment = "center",
                         path_effects=[pe.withStroke(linewidth = 4, foreground = "white",alpha = 0.2)])
        else:
            ax4.annotate(f"{strategy_df[strategy_df.index == Ticker]['returns_(%)'][0]} %",
                         annotation_clip = False, xy=(0.05,-0.25),
                         font = letra,
                         fontsize = 40,
                         color = "red",multialignment = "center",
                         alpha = 0.8,
                         path_effects=[pe.withStroke(linewidth = 4, foreground = "white",alpha = 0.2)])
            
            ax4.annotate(f"€ {int(strategy_df[strategy_df.index == Ticker]['returns_(eur)'][0])}", annotation_clip = False, xy=(0.05,0.35),
                         font = letra,
                         fontsize = 50,
                         color = "red",multialignment = "center",
                         path_effects=[pe.withStroke(linewidth = 4, foreground = "white",alpha = 0.2)])

        ax4.set_title("Returns", font = letra, fontweight = "bold", fontsize=28, color = "white", y = 0.9)
        ax4.set_axis_off()
        
    if ax5:
#         ax5.patch.set_facecolor('#00193E')
#         ax5.scatter(data2[Ticker]['Close'].rolling(30).std(),
#                     returns)
        
#         ax5.set_xlabel(f"Volatility 30d [%]",color="w")
#         ax5.set_ylabel("Returns",color="w")         
#         ax5.spines[:].set_visible(False)
        # ax5.grid(False)
        ax5.set_title("30 last days Volume", font = letra, fontweight = "bold", fontsize=20, color = "white", y = 1)
        up = data2[Ticker][-30:][data2.loc[:,(Ticker,"Close")] >= data2.loc[:,(Ticker,"Open")]]
        down = data2[Ticker][-30:][data2.loc[:,(Ticker,"Close")] < data2.loc[:,(Ticker,"Open")]]
        
        ax5.bar(up.index, up.Volume, width, color='green', edgecolor="#00193E")
        ax5.bar(down.index, down.Volume, width, color='red', edgecolor="#00193E")
        ax5.set_ylabel("Volume",font = letra, fontsize=15)
        ax5.grid(False)
        
        data2[Ticker][-30:].Volume.diff().plot(ax=ax5, c="yellow", linestyle = "dashed", linewidth = 2, label = "Volume diff")
        ax5.axhline(data2[Ticker][-30:].Volume.mean(), c="w", linestyle = "dashed", linewidth = 2, label = "Volume mean")
        ax5.get_xaxis().set_visible(False)
        ax5.get_yaxis().set_visible(False)
        ax5.set_axis_off()
        ax5.legend(loc= (0.58,-0.33))
        
        ax51 = ax5.twinx()
        ax51.plot(data2[(Ticker,"Close")].diff(5), c="lightblue", label="Momentum", linewidth=1)
        ax51.legend(loc= (0,-0.2))
        ax51.grid(False)
        ax51.set_axis_off()
        
    if ax6:
        ax6.patch.set_facecolor('#00193E')
        ax6.barh(["Portfolio", "Total","Period"], [volatility_cartera,volat_total,volat],
                  color = ["brown","red","orange"], edgecolor = '#00193E',
                 alpha = 0.5)
        ax6.spines[:].set_visible(False)
        # ax6.set_xticks([0, 25, 50, 75, 100])
        ax6.get_xaxis().set_visible(False)
        ax6.grid(False)
        ax6.set_xmargin(0.5)
        ax6.set_title("Risk", font = letra, fontsize = 28, fontweight = "bold", color = "w", pad = 5)

        # ax5.annotate(f"Period/global", xy = (volat/volat_total*100-5, -0.3), fontsize = 18, annotation_clip = False)
        ax6.annotate(f"{volat}%", xy = (np.max([volat,volat_total,volatility_cartera])+0.1*np.max([volat,volat_total,volatility_cartera]), 1.8), 
                     fontsize = 25, color = "w",
                    path_effects=[pe.withStroke(linewidth = 3, foreground = "white",alpha = 0.1)], annotation_clip = False)
        
        ax6.annotate(f"{volat_total}%", xy = (np.max([volat,volat_total,volatility_cartera])+0.1*np.max([volat,volat_total,volatility_cartera]), 0.8)
                     , fontsize = 25, color = "w",
                    path_effects=[pe.withStroke(linewidth = 3, foreground = "white",alpha = 0.1)], annotation_clip = False) 
        
        ax6.annotate(f"{volatility_cartera}%", xy = (np.max([volat,volat_total,volatility_cartera])+0.1*np.max([volat,volat_total,volatility_cartera]), -0.2),
                     fontsize = 25, color = "w",
                    path_effects=[pe.withStroke(linewidth = 3, foreground = "white",alpha = 0.1)], annotation_clip = False) 
        
    plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.2, 
                    top=0.5, 
                    wspace=0.6, 
                    hspace=0.8)
    fig.tight_layout()
    plt.show()

Ticker = widgets.Dropdown(options = analysis_ticks, description = "Ticker",
                     style = {'description_width': 'initial'}, 
                          layout=widgets.Layout(width="12%"))

if inter2 == 3:
    start5 = widgets.IntSlider(min = -len(analysis_df.resample(f'1d')), max = -10, step = 30, value = -365, description = "Steps before",
                                      style = {'description_width': 'initial'},
                                     layout=widgets.Layout(width="30%"))
elif inter2 == 2:
    start5 = widgets.IntSlider(min = -len(analysis_df.resample(f'1h')), max = -1, step = 1, value = -5, description = "Steps before",
                                      style = {'description_width': 'initial'},
                                     layout=widgets.Layout(width="30%"))
else:
    start5 = widgets.IntSlider(min = -len(analysis_df.resample(f'5m')), max = -1, step = 1, value = -1, description = "Steps before",
                                      style = {'description_width': 'initial'},
                                     layout=widgets.Layout(width="30%"))

step1 = widgets.IntSlider(min = 2, max = 22, step = 4 , value = 10, description = "Step 1", 
                          layout=widgets.Layout(width="23%"))
step2 = widgets.IntSlider(min = 10, max = 365, step = 10 , value = 40, description = "Step 2", 
                          layout=widgets.Layout(width="23%"))

t = widgets.Dropdown(options = ["d","h","m"], description = "Prices Interval",
                     style = {'description_width': 'initial'}, 
                     layout=widgets.Layout(width="12%"))

ui = widgets.Box([Ticker, start5, step1, step2 ,t])
out = widgets.interactive_output(ma_cb_plot, {'Ticker': Ticker, 'start5': start5, 'step1':step1, 'step2':step2, 't': t})
display(ui,out)

Box(children=(Dropdown(description='Ticker', layout=Layout(width='12%'), options=('AAPL', 'BMW.DE', 'BTC-USD',…

Output()

### 4. TESTS

In [16]:
# plt.bar(x = returns[start:].describe().iloc[2,:].index, height = returns[start:].describe().iloc[2,:])

In [17]:
# returns[start:].describe().iloc[2,:].values

In [18]:
# sc = StandardScaler()
# std_sc = sc.fit_transform(returns[start:].describe().iloc[2,:].values.reshape[-1,1])

In [19]:
# colors = sns.color_palette('pastel')
# plt.pie(returns[start:].describe().iloc[2,:], colors = colors)
# plt.show()

In [20]:
# strategy_df

In [21]:
# def dist_plot(Ticker, start = -120):
#         sns.distplot(returns.loc[:,Ticker][start:])
#         plt.show()

# interact(dist_plot, Ticker = analysis_ticks,
#         start = widgets.IntSlider(min = -360, max = -10, step = 15, value = -260))

In [22]:
# def desc_returns(start,Ticker):
#     df = np.round(returns.iloc[start:,].describe().iloc[1:,:]*100,2)
    
#     fig, ax = plt.subplots(figsize = (10,5))
#     ax.bar(x = df.loc['std',analysis_ticks].sort_values().index, 
#            height = df.loc['std',analysis_ticks].sort_values(), 
#            color= "red",
#            alpha=0.4,
#            edgecolor='black',
#            )
    
#     ax.set_title(f"Volatility last {-start} days", fontsize = 23, pad = 15)
    
#     ax.set_ylabel("Returns's std [%]")
    
#     # sns.distplot(returns.loc[:,Ticker][start:])
#     plt.show()

# interact(desc_returns, Ticker = analysis_ticks,
#          start = widgets.IntSlider(min = -360, max = -10, step = 15, value = -260))

In [23]:
# # VOLATILITY - RISK MEASURE
# np.round(data2.loc["2021-03":,(analysis_ticks,"returns")].std()*100,2)

In [24]:
# corr_returns = data2.loc[:,(analysis_ticks,"returns")].droplevel(level = 1, axis=1).corr()
# corr_returns = returns.corr()

In [25]:
# fig,ax = plt.subplots(figsize=(12,8))
# sns.heatmap(corr_returns, annot=True,cmap= 'coolwarm',annot_kws={'size': 12}, ax = ax)
# ax.set_title("Returns", fontsize = 25, pad = 20)

# plt.show()

In [26]:
# sns.pairplot(returns)
# plt.show()

#### 4.0 INDICATORS

In [27]:
# recom_0 = yf.Ticker(analysis_ticks[0]).recommendations
# recom_0["To Grade"]

In [28]:
# np.round(recom_0["To Grade"].value_counts()/recom_0["To Grade"].value_counts().sum()*100,2)

In [29]:
# tic = inv.search_quotes(analysis_ticks[0], n_results=1)
# tic.retrieve_technical_indicators(interval="daily")

In [30]:
# color_palette = sns.color_palette("vlag", as_cmap=True)
# np.round(data2.loc["2022-03":,(analysis_ticks,"Close")],2).style.background_gradient(color_palette)

In [31]:
# returns_d["2022-05"].style.bar(color = ['#d65f5f', '#5fba7d'], align = 'mid')

$\mathbf{\text{EPS = Earnings per share}}$<br>

1.&emsp;Earnings per share (EPS) is calculated as a company's profit divided by the outstanding shares of its common stock. The resulting number serves as an indicator of a company's profitability. It is common for a company to report EPS that is adjusted for extraordinary items and potential share dilution.
<br>

$$ EpS = \frac{End of Period Common Shares Outstanding - Net Income}{Preferred Dividends} $$

In [32]:
# si.get_cash_flow("MSFT")

In [33]:
# si.get_company_info("MSFT")

In [34]:
# si.get_analysts_info("MSFT")

In [35]:
# si.get_earnings_history